# Análisis de patrones y preferencias de los clientes de taxis de Chicago para Zuber

# Contenido

* [Introducción](#)
* [Objetivos](#)
* [Etapas](#)
* [Inicialización](#)
    * [Cargar los datos](#)
* [Preprocesamiento de datos](#)
* [Análisis exploratorio y descriptivo de datos](#)
* [Pruebas de hipótesis](#)
* [Conclusiones](#)

# Introducción

Una empresa de viajes compartidos, Zuber, está lanzando sus servicios en la ciudad de Chicago. Zuber requiere analizar los diversos patrones de comportamiento de los usuarios de esta ciudad y analizar si los factores externos afectan significativamente a los viajes.

Para este trabajo primero recuperamos información meteorológica de la ciudad de Chicago desde una página web utilizando técnicas de minería web. Luego, se logró tener acceso a las bases de datos relacionales de la empresa. Utilizamos el gestor de bases de datos relacionales PostgreSQL y realizando distintas operaciones con el lenguaje SQL sintetizamos toda esta información en tres datasets con los que trabajaremos para lograr los objetivos que se presentarán en el siguiente apartado.

# Objetivos

La empresa requiere identificar a los 10 principales barrios y empresas de taxis, respectivamente. Además, de manera más rigurosa, quiere que probemos la siguiente hipótesis en particular:
- La duración promedio de los viajes desde Loop hasta el Aeropuerto Internacional O'Hare cambia los sábados lluviosos.

# Etapas del análisis

En el desarrollo de este proyecto, seguiremos el siguiente esquema básico para mantener el proceso ordenado:

1. Inicialización: descripción de datos
2. Preprocesamiento de datos
3. Análisis exploratorio y descriptivo de datos
4. Prueba de hipótesis
5. Conclusiones

# Inicialización: descripción de los datos

**Carga de librerías necesarias**

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats as st 
# carga todas las librerías necesarias

## Cargar los datos

In [4]:
try:
    data_company = pd.read_csv('moved_project_sql_result_01.csv')
    data_neighb = pd.read_csv('moved_project_sql_result_04.csv')
    data_weather = pd.read_csv('moved_project_sql_result_07.csv')
except:
    data_company = pd.read_csv('/datasets/project_sql_result_01.csv')
    data_neighb = pd.read_csv('/datasets/project_sql_result_04.csv')
    data_weather = pd.read_csv('/datasets/project_sql_result_07.csv')

## Exploración inicial de datos

Comenzaremos con la exploración de la tabla ``data_company``.

**Exploración de la tabla `data_company`**

Esta tabla, resultado de la agrupación de información de las bases de datos para los días 15 y 16 de noviembre de 2017, cuenta con la siguiente información:
* ``company_name``: nombre de la empresa de taxis.
* ``trips_amount``: número de viajes de cada compañía de taxis.

Ahora obtendremos su información:

In [5]:
data_company.describe()

,trips_amount
count,64.000000
mean,2145.484375
std,3812.310186
min,2.000000
25%,20.750000
50%,178.500000
75%,2106.500000
max,19558.000000


In [8]:
data_company.head(10)

,company_name,trips_amount
0,Flash Cab,19558
1,Taxi Affiliation Services,11422
2,Medallion Leasin,10367
3,Yellow Cab,9888
4,Taxi Affiliation Service Yellow,9299
5,Chicago Carriage Cab Corp,9181
6,City Service,8448
7,Sun Taxi,7701
8,Star North Management LLC,7455
9,Blue Ribbon Taxi Association Inc.,5953


In [9]:
data_company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


Analizando por columnas, tenemos lo siguiente:
* ``company_name`` no parece mostrar problemas, ni ausentes. Es una columna categórica.
* ``trips_amount`` no parece tener problemas ni ausentes. Aunque si nos fijamos en su descripción, observamos un gran sesgo positivo de su distribución: su media es bastante más grande que su mediana.

Echaremos un vistazo a la columna categórica de este dataset:

In [14]:
print("Valores únicos de la columna 'company_name':")
data_company['company_name'].unique()

Valores únicos de la columna 'company_name':


array(['Flash Cab', 'Taxi Affiliation Services', 'Medallion Leasin',
       'Yellow Cab', 'Taxi Affiliation Service Yellow',
       'Chicago Carriage Cab Corp', 'City Service', 'Sun Taxi',
       'Star North Management LLC', 'Blue Ribbon Taxi Association Inc.',
       'Choice Taxi Association', 'Globe Taxi',
       'Dispatch Taxi Affiliation', 'Nova Taxi Affiliation Llc',
       'Patriot Taxi Dba Peace Taxi Associat', 'Checker Taxi Affiliation',
       'Blue Diamond', 'Chicago Medallion Management', '24 Seven Taxi',
       'Chicago Medallion Leasing INC', 'Checker Taxi', 'American United',
       'Chicago Independents', 'KOAM Taxi Association', 'Chicago Taxicab',
       'Top Cab Affiliation', 'Gold Coast Taxi',
       'Service Taxi Association', '5 Star Taxi', '303 Taxi',
       'Setare Inc', 'American United Taxi Affiliation', 'Leonard Cab Co',
       'Metro Jet Taxi A', 'Norshore Cab', '6742 - 83735 Tasha ride inc',
       '3591 - 63480 Chuks Cab', '1469 - 64126 Omar Jada',
       '6

No parecen existir problemas más allá de que sus valores usan mayúsculas en sus nombres. Ya que la agrupación la realizamos en SQL, no necesitamos obtener la distribución de valores, porque la tabla lo muestra en sí. 

Pasaremos a la siguiente columna.

**Exploración de la tabla ``data_neighb``**

Esta tabla también proviene de la manipulación de las bases de datos, en este caso mostrando los barrios y sus viajes en el mes de noviembre de 2017, con las siguientes columnas:
* ``dropoff_location_name``: barrios de Chicago donde finalizaron los viajes.
* ``average_trips``: promedio de viajes que terminaron en el respectivo barrio.

Exploremos este dataset:

In [15]:
data_neighb.describe()

,average_trips
count,94.000000
mean,599.953728
std,1714.591098
min,1.800000
25%,14.266667
50%,52.016667
75%,298.858333
max,10727.466667


In [16]:
data_neighb.head(10)

,dropoff_location_name,average_trips
0,Loop,10727.466667
1,River North,9523.666667
2,Streeterville,6664.666667
3,West Loop,5163.666667
4,O'Hare,2546.900000
5,Lake View,2420.966667
6,Grant Park,2068.533333
7,Museum Campus,1510.000000
8,Gold Coast,1364.233333
9,Sheffield & DePaul,1259.766667


In [17]:
data_neighb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB


Observando por columnas, tenemos:
* ``dropoff_location_name`` no parece mostrar problemas ni valores ausentes; es una variable categórica.
* ``average_trips`` no parece tener ausentes, aunque nuevamente notamos un gran sesgo en su distribución, siendo su media bastante más elevada que su mediana. Quizá sea recomendable redondear los valores promedio para no tener problemas en las interpretaciones y hacer el análisis más explicable. 

Exploraremos un poco más sobre la variable categórica de este dataset:

In [18]:
print("Valores únicos de la columna 'dropoff_location_name':")
data_neighb['dropoff_location_name'].unique()

Valores únicos de la columna 'dropoff_location_name':


array(['Loop', 'River North', 'Streeterville', 'West Loop', "O'Hare",
       'Lake View', 'Grant Park', 'Museum Campus', 'Gold Coast',
       'Sheffield & DePaul', 'Lincoln Park', 'East Village',
       'Little Italy, UIC', 'Uptown', 'Near South Side', 'Garfield Ridge',
       'Logan Square', 'Edgewater', 'West Town', 'Old Town',
       'Rush & Division', 'North Center', 'Lincoln Square', 'Rogers Park',
       'West Ridge', 'Irving Park', 'Hyde Park', 'Avondale',
       'Wicker Park', 'Albany Park', 'United Center', 'Lower West Side',
       'Douglas', 'Portage Park', 'Humboldt Park', 'Norwood Park',
       'Kenwood', 'Bridgeport', 'Armour Square', 'Jefferson Park',
       'Bucktown', 'North Park', 'Garfield Park', 'Mckinley Park',
       'Belmont Cragin', 'Boystown', 'Chinatown', 'Grand Boulevard',
       'Austin', 'Sauganash,Forest Glen', 'South Shore', 'Woodlawn',
       'Little Village', 'Jackson Park', 'North Lawndale', 'Dunning',
       'Ukrainian Village', 'Hermosa', 'Englewood'

No observamos problemas más allá del uso de mayúsculas en los nombres. De la misma forma que en el dataset anterior, ya que estos datos son agrupaciones obtenidas desde la base de datos, nos muestra la distribución de sus respectivos valores y ya no es necesario mostrarlo. 

Pasamos al último dataset.

**Exploración de la tabla ``data_weather``**

Este dataset combina datos obtenidos de una página web sobre información meteorológica de la ciudad de Chicago, el cual unimos con otras tablas de la base de datos para obtener la siguiente información:

* ``start_ts``: fecha y hora de la recogida, tomamos datos de solo los días sábados.
* ``weather_conditions``: condiciones climáticas en el momento en el que comenzó el viaje.
* ``duration_seconds``: duración del viaje en segundos.

Comenzaremos con la exploración:

In [19]:
data_weather.describe()

,duration_seconds
count,1068.000000
mean,2071.731273
std,769.461125
min,0.000000
25%,1438.250000
50%,1980.000000
75%,2580.000000
max,7440.000000


In [20]:
data_weather.head(10)

,start_ts,weather_conditions,duration_seconds
0,2017-11-25 16:00:00,Good,2410.0
1,2017-11-25 14:00:00,Good,1920.0
2,2017-11-25 12:00:00,Good,1543.0
3,2017-11-04 10:00:00,Good,2512.0
4,2017-11-11 07:00:00,Good,1440.0
5,2017-11-11 04:00:00,Good,1320.0
6,2017-11-04 16:00:00,Bad,2969.0
7,2017-11-18 11:00:00,Good,2280.0
8,2017-11-11 14:00:00,Good,2460.0
9,2017-11-11 12:00:00,Good,2040.0


In [21]:
data_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   start_ts            1068 non-null   object 
 1   weather_conditions  1068 non-null   object 
 2   duration_seconds    1068 non-null   float64
dtypes: float64(1), object(2)
memory usage: 25.2+ KB


Analizando por columnas, tenemos:

* ``start_ts`` es de tipo fecha, sin embargo esto no está registrado correctamente. No se tienen valores ausentes en esta columna.
* ``weather_conditions`` es una variable dicotómica, con dos valores posibles que se le dieron en la obtención de datos desde la base de datos: "Good" y "Bad". No parece tener problemas ni valores ausentes.
* ``duration_seconds`` no parece mostrar problemas, aunque viendo su distribución notamos cierto sesgo y valores atípicos como ceros (viajes de 0 seg de duración), lo cual no parece tener mucho sentido. Analizaremos esto en el preprocesamiento siguiente.

Para confirmar lo dicho sobre nuestra variable categórica, mostraremos sus valores únicos:

In [22]:
print("Valores únicos de la columna 'weather_conditions':")
data_weather['weather_conditions'].unique()

Valores únicos de la columna 'weather_conditions':


array(['Good', 'Bad'], dtype=object)

Al igual que en los datasets previos, no se presentan problemas graves en los datos de esta tabla. En este caso podemos mostrar la distribución de valores para cada condición del tiempo que se tiene para complementar el análisis:

In [24]:
data_weather['weather_conditions'].value_counts(dropna=False)

Good    888
Bad     180
Name: weather_conditions, dtype: int64

Observamos que nuestra información detalla que la mayor parte del tiempo de los sábados de noviembre en Chicago tienen condiciones climáticas favorables para el desarrollo y operatividad de la empresa. 

## Conclusiones del apartado

Podemos concluir lo siguiente luego de haber explorado nuestros datos:

* En general, nuestros datasets no cuentan con problemas de valores ausentes ni errores.
* Las tres tablas, en sus respectivos valores numéricos, presentan importantes sesgos en sus datos, evidenciando la existencia de valores atípicos que necesitan ser tratados en los siguientes apartados.
* La tabla ``data_weather`` cuenta con una variable de tipo fecha que necesita ser corregida para facilitar su análisis.

# Preprocesamiento de datos

Realizaremos el proceso para cada tabla, basándonos en las conclusiones mostradas del último apartado. Debido a que ``data_company`` no muestra errores para corregir en este apartado, pasaremos a la tabla ``data_neighb``.

## Corregir datos de ``data_neighb``

La única recomendación en esta tabla para faciltar el análisis que dimos es la posibilidad de redondear sus valores numéricos, ya que cuentan con demasiados decimales y dificulta la interpretación. Por naturaleza, un viaje debe ser de tipo entero y realizaremos este proceso de corrección:

In [26]:
data_neighb['average_trips'] = data_neighb['average_trips'].round().astype('int')
data_neighb['average_trips']

0     10727
1      9524
2      6665
3      5164
4      2547
      ...  
89        3
90        3
91        2
92        2
93        2
Name: average_trips, Length: 94, dtype: int32

Una vez solucionado, pasamos a la última tabla.

## Corregir datos de ``data_weather``